In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
import hopsworks
import pandas as pd
from datetime import timedelta
from src.inference import get_feature_store, fetch_predictions, fetch_hourly_rides


In [11]:
df = fetch_hourly_rides(24)

2025-05-09 01:42:13,079 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-09 01:42:13,100 INFO: Initializing external client
2025-05-09 01:42:13,102 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-05-09 01:42:14,450 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213682
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.35s) 


In [12]:
df

,start_hour,start_station_id,rides
0,2025-05-08 02:00:00-04:00,6230.02,6
1,2025-05-08 00:00:00-04:00,5308.04,0
2,2025-05-08 14:00:00-04:00,5308.04,29
3,2025-05-08 02:00:00-04:00,5872.10,2
4,2025-05-08 22:00:00-04:00,5308.04,0
...,...,...,...
73,2025-05-08 10:00:00-04:00,5872.10,9
74,2025-05-08 11:00:00-04:00,5308.04,18
75,2025-05-08 14:00:00-04:00,6230.02,10
76,2025-05-08 08:00:00-04:00,5872.10,9


In [7]:
df_pred = fetch_predictions(24)

2025-05-09 01:40:11,157 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-09 01:40:11,173 INFO: Initializing external client
2025-05-09 01:40:11,173 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-05-09 01:40:12,443 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213682
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.39s) 


In [8]:
df_pred

,start_station_id,predicted_demand,start_hour
0,6230.02,5.0,2025-05-09 02:00:00-04:00
1,5872.10,5.0,2025-05-09 02:00:00-04:00
2,5308.04,9.0,2025-05-09 02:00:00-04:00


In [13]:
merged_df = pd.merge(df, df_pred, on=['start_station_id', 'start_hour'])

In [14]:
merged_df

,start_hour,start_station_id,rides,predicted_demand


In [ ]:
import pandas as pd  
import plotly.express as px
df1 = df
df2 = df_pred

# Merge the DataFrames on 'pickup_location_id' and 'pickup_hour'  
merged_df = pd.merge(df1, df2, on=['pickup_location_id', 'pickup_hour'])  

# Calculate the absolute error  
merged_df['absolute_error'] = abs(merged_df['predicted_demand'] - merged_df['rides'])  

# Group by 'pickup_hour' and calculate the mean absolute error (MAE)  
mae_by_hour = merged_df.groupby('pickup_hour')['absolute_error'].mean().reset_index()  
mae_by_hour.rename(columns={'absolute_error': 'MAE'}, inplace=True)  

# Create a Plotly plot  
fig = px.line(  
    mae_by_hour,  
    x='pickup_hour',  
    y='MAE',  
    title='Mean Absolute Error (MAE) by Pickup Hour',  
    labels={'pickup_hour': 'Pickup Hour', 'MAE': 'Mean Absolute Error'},  
    markers=True  
)  

# Show the plot  
fig.show()

In [ ]:
mae_by_hour["MAE"].mean()